In [1]:
import os
%pwd

os.chdir('../')
%pwd

'c:\\Users\\theVa\\Documents\\GitHub\\mlops-course\\7-EndToEnd'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: str
    train_data_path: str
    test_data_path: str 
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [20]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_fielpath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_fielpath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params
        schema = self.schema

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.ElasticNet.alpha,
            l1_ratio=params.ElasticNet.l1_ratio,
            target_column=schema.TARGET_COLUMN.name
        )
        return model_trainer_config

In [ ]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.configManager = config

    def train(self):
        train_data = pd.read_csv(self.configManager.train_data_path)
        test_data = pd.read_csv(self.configManager.test_data_path)

        X_train = train_data.drop(columns=[self.configManager.target_column])
        Y_train = train_data[[self.configManager.target_column]]

        X_test = test_data.drop(columns=[self.configManager.target_column])
        Y_test = test_data[[self.configManager.target_column]]

        model = ElasticNet(alpha=self.configManager.alpha,
                            l1_ratio=self.configManager.l1_ratio,
                        )
        model.fit(X_train,Y_train)

        joblib.dump(model,os.path.join(self.configManager.root_dir,self.configManager.model_name))
        print('Finished Saving trained model')


In [35]:
try:
    configManager = ConfigurationManager()
    config = configManager.get_model_trainer_config()
    trainer = ModelTrainer(config = config)
    trainer.train()
except Exception as e:
    raise e

[2026-01-03 12:54:49,035: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2026-01-03 12:54:49,037: INFO: common: yaml file: params.yaml loaded successfully.]
[2026-01-03 12:54:49,040: INFO: common: yaml file: schema.yaml loaded successfully.]
[2026-01-03 12:54:49,041: INFO: common: Created directory at artifacts]
[2026-01-03 12:54:49,042: INFO: common: Created directory at artifacts/model_trainer]
